# Please install the following python libraries
- python3: https://www.python.org/
- numpy: https://numpy.org/install/
- tqdm: https://github.com/tqdm/tqdm#installation
- matplotlib: https://matplotlib.org/stable/users/installing/index.html

If you encounter the error: "IProgress not found. Please update jupyter & ipywidgets"
    
Please install the ipywidgets as follows:

    with pip, do
    - pip install ipywidgets
    
    with conda, do
    - conda install -c conda-forge ipywidgets
    
Restart your notebook 

## Please install the Pytorch library on your computer before you run this notebook.

The installation instructions can be found here. (https://pytorch.org/get-started/locally/)

In this exercise, we will use the Pytorch library (https://pytorch.org/) to build and train our deep neural networks. In the deep learning literature, especially in the research community, Pytorch is SUPER popular due to its automatic differentiation and dynamic computational graph (i.e., the graph is automatically generated, which is different from tensorflow where you have to define them beforehand). Briefly spearking, using Pytorch, you only have to build your neural network, define the forward pass, and the loss function. The library will automatically compute the weights and perform the backpropagation for you. For more details about Pytorch, we recommend you check the tutorails on the offical website to learn the basics (https://pytorch.org/tutorials/). 

Please try to learn the basics as much as you can. If you have any questions, feel free to ask them on Piazza or TA hours. 

In [ ]:
import numpy as np
import tqdm
import torch
from torch import nn
from torch.distributions import Categorical
import torch.nn.functional as F
import matplotlib.pyplot as plt
from collections import deque
from gym import spaces
import gym

# Four Rooms environment

Note that, to make the environment similar to environments in gym, we change the action space to be [0, 1, 2, 3] using the gym.spaces.Discrete(4) where we can use it to sample a random action. Please read the code below to make sure you understand the changes.

Besides, we change the reward function back to the original one. Specifically, the agent receives 1 for reaching the goal but 0 otherwise.

Finally, we recommend you to start with the provided hyperparameters to debug.

The environment and plotting function are provided. You do have to implement it but feel free to change to 
adapt them to your implementation.

In [ ]:
class FourRooms(object):
    def __init__(self):
        # We define the grid for the Four Rooms domain
        self.grid = np.array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                              [1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1],
                              [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

        # We define the observation space consisting of all empty cells
        # Note: We have to flip the coordinates from (row_idx, column_idx) -> (x, y),
        # where x = column_idx, y = 10 - row_idx
        self.observation_space = np.argwhere(self.grid == 0.0).tolist()  # Fine all empty cells
        self.observation_space = self.arr_coords_to_four_room_coords(self.observation_space)

        """ Note that, we change the action space as follows
        """
        # We define the action space
        self.action_movement = {0: np.array([0, 1]),  # up
                                1: np.array([0, -1]),  # down
                                2: np.array([-1, 0]),  # left
                                3: np.array([1, 0])}  # right
        self.action_space = spaces.Discrete(4)

        # We define the start location
        self.start_location = [0, 0]

        # We define the goal location
        self.goal_location = [10, 10]

        # We find all wall cells
        self.walls = np.argwhere(self.grid == 1.0).tolist()  # find all wall cells
        self.walls = self.arr_coords_to_four_room_coords(self.walls)  # convert to Four Rooms coordinates

        # This is an episodic task, we define a timeout: maximal time steps = 459
        self.max_time_steps = 459

        # We define other useful variables
        self.agent_location = None  # track the agent's location in one episode.
        self.action = None  # track the agent's action
        self.t = 0  # track the current time step in one episode

    @staticmethod
    def arr_coords_to_four_room_coords(arr_coords_list):
        """
        Function converts the array coordinates to the Four Rooms coordinates (i.e, The origin locates at bottom left).
        E.g., The coordinates (0, 0) in the numpy array is mapped to (0, 10) in the Four Rooms coordinates.
        Args:
            arr_coords_list (list): a list variable consists of tuples of locations in the numpy array

        Return:
            four_room_coords_list (list): a list variable consists of tuples of converted locations in the
                                          Four Rooms environment.
        """
        # Note: We have to flip the coordinates from (row_idx, column_idx) -> (x, y),
        # where x = column_idx, y = 10 - row_idx
        four_room_coords_list = [(column_idx, 10 - row_idx) for (row_idx, column_idx) in arr_coords_list]
        return four_room_coords_list

    def reset(self):
        # We reset the agent's location to the start location
        self.agent_location = self.start_location

        # We reset the timeout tracker to be 0
        self.t = 0

        # We set the information
        info = {}
        return self.agent_location, info

    def step(self, action):
        """
        Args:
            action (int): a int variable (i.e., 0 for "up"). See the self.action_movement above for more details
        """
        # With probability 0.8, the agent takes the correct direction.
        # With probability 0.2, the agent takes one of the two perpendicular actions.
        # For example, if the correct action is "LEFT", then
        #     - With probability 0.8, the agent takes action "LEFT";
        #     - With probability 0.1, the agent takes action "UP";
        #     - With probability 0.1, the agent takes action "DOWN".
        if np.random.uniform() < 0.2:
            if action == 2 or action == 3:
                action = np.random.choice([0, 1], 1)[0]
            else:
                action = np.random.choice([2, 3], 1)[0]

        # Convert the agent's location to array
        loc_arr = np.array(self.agent_location)

        # Convert the action name to movement array
        act_arr = self.action_movement[action]

        # Compute the agent's next location
        next_agent_location = np.clip(loc_arr + act_arr,
                                      a_min=np.array([0, 0]),
                                      a_max=np.array([10, 10])).tolist()

        # Check if the agent crashes into walls, it stays at the current location.
        if tuple(next_agent_location) in self.walls:
            next_agent_location = self.agent_location

        """ Note that, we change the reward back to the 
            original one. 
            1 for reaching the goal
            0 otherwise
        """
        # Compute the reward
        reward = 1.0 if next_agent_location == self.goal_location else 0.0

        # Check the termination
        # If the agent reaches the goal, reward = 1, done = True
        # If the time steps reaches the maximal number, reward = 0, done = True.
        if reward == 1.0 or self.t == self.max_time_steps:
            terminated = True
        else:
            terminated = False

        # Update the agent's location, action and time step trackers
        self.agent_location = next_agent_location
        self.action = action
        self.t += 1

        return next_agent_location, reward, terminated, False, {}

    def render(self):
        # plot the agent and the goal
        # empty cell = 0
        # wall cell = 1
        # agent cell = 2
        # goal cell = 3
        plot_arr = self.grid.copy()
        plot_arr[10 - self.agent_location[1], self.agent_location[0]] = 2
        plot_arr[10 - self.goal_location[1], self.goal_location[0]] = 3
        plt.clf()
        plt.title(f"state={self.agent_location}, act={self.action_movement[self.action]}")
        plt.imshow(plot_arr)
        plt.show(block=False)
        plt.pause(0.1)

    @staticmethod
    def test():
        my_env = FourRooms()
        state, _ = my_env.reset()

        for _ in range(100):
            action = np.random.choice(list(my_env.action_space.keys()), 1)[0]

            next_state, reward, done, _, _ = my_env.step(action)
            my_env.render()

            if done:
                state, _ = my_env.reset()
            else:
                state = next_state

In [ ]:
def plot_curves(arr_list, legend_list, color_list, ylabel, fig_title):
    """
    Args:
        arr_list (list): list of results arrays to plot
        legend_list (list): list of legends corresponding to each result array
        color_list (list): list of color corresponding to each result array
        ylabel (string): label of the Y axis

        Note that, make sure the elements in the arr_list, legend_list and color_list are associated with each other correctly.
        Do not forget to change the ylabel for different plots.
    """
    # set the figure type
    fig, ax = plt.subplots(figsize=(12, 8))

    # PLEASE NOTE: Change the labels for different plots
    ax.set_ylabel(ylabel)
    ax.set_xlabel("Time Steps")

    # ploth results
    h_list = []
    for arr, legend, color in zip(arr_list, legend_list, color_list):
        # compute the standard error
        arr_err = arr.std(axis=0) / np.sqrt(arr.shape[0])
        # plot the mean
        h, = ax.plot(range(arr.shape[1]), arr.mean(axis=0), color=color, label=legend)
        # plot the confidence band
        arr_err *= 1.96
        ax.fill_between(range(arr.shape[1]), arr.mean(axis=0) - arr_err, arr.mean(axis=0) + arr_err, alpha=0.3,
                        color=color)
        # save the plot handle
        h_list.append(h)

    # plot legends
    ax.set_title(f"{fig_title}")
    ax.legend(handles=h_list)

    plt.show()

# Q3 : Implement the REINFORCE algorithm on page 328

Here, we will implement the REINFORCE algorithm on page 328 using the neural networks. 

Same as the ex8, we still use the Pytorch library to build the neural networks and train them.

Please read the pseudocode on page 328 carefully and complete the implementation below.

## Model the policy using a fully connected neural network

Here, you are asked to implement a neural network to approximate the stochastic policy \pi(a|s). In particular, the architecture of the neural network is as follows:

Layer 1: Linear, input size 3, output size 128

Activation 1: relu

Layer 2: Linear, input size 128, output size 4

Activation 2: softmax

Note that, instead of using the original state = [x, y], we represent each state as [x / 10, y / 10, 1]. In other words, we use a simple normalized feature.  

In [ ]:
class PolicyNet(nn.Module):
    def __init__(self):
        super(PolicyNet, self).__init__()

        """ CODE HERE:
                Implement the neural network here
        """
        self.network = None

    def forward(self, x):
        """ CODE HERE:
                Implement the forward propagation
        """
        y = None
        return y

## Implement the REINFORCE agent as a class

We implement the REINFORCE agent as a class. Specifically, we model the policy as a fully connected neural networks. 

Besides, we implement a function to derive an action from the neural network

In [ ]:
class REINFORCEAgent(object):
    def __init__(self):
        # create the policy network
        self.policy_net = PolicyNet()

    def get_action(self, state):
        """ Function to derive an action given a state
            Args:
                state (list): [x/10, y/10, 1]
                
            Returns:
                action index (int), log_prob (ln(\pi(action|state)))
        """
        state_tensor = torch.tensor(state).float().view(1, -1)
        probs = self.policy_net(state_tensor)
        m = Categorical(probs)
        action = m.sample()
        log_prob = m.log_prob(action)
        return action.item(), log_prob

## Implement the REINFORCE algorithm as a class

In [ ]:
def rolling_average(data, window_size):
    """Smoothen the 1-d data array using a rollin average.

    Args:
        data: 1-d numpy.array
        window_size: size of the smoothing window

    Returns:
        smooth_data: a 1-d numpy.array with the same size as data
    """
    assert data.ndim == 1
    kernel = np.ones(window_size)
    smooth_data = np.convolve(data, kernel) / np.convolve(
        np.ones_like(data), kernel
    )
    return smooth_data[: -window_size + 1]

In [ ]:
class REINFORCEAgentTrainer(object):
    def __init__(self, agent, env, params):
        # Agent object
        self.agent = agent
        
        # Environment object
        self.env = env
        
        # Training parameters
        self.params = params

        # Lists to store the log probabilities and rewards for one episode
        self.saved_log_probs = []
        self.saved_rewards = []

        # Gamma
        self.gamma = params['gamma']
        
        # Small value for returns normalization
        self.eps = params['epsilon']

        # Create the optimizer
        """ CODE HERE:
                Implement the Adam optimizer with the learning rate in params
        """

    @staticmethod
    def compute_state_feature(state):
        return [state[0] / 10, state[1] / 10, 1]

    def update_agent_policy_network(self):
        # We define a list to store the policy loss for each time step
        policy_loss = []
        
        # We define a special list to store the return for each time step
        returns = deque()

        """ CODE HERE:
                Compute the return for each time step
                
                Hint: We usually compute the return from the end. Remember to append it 
                      correctly. You can use "returns.appendleft(G)"
        """
        # compute returns for every time step


        # normalize the returns: for stablize the training
        returns = torch.tensor(returns)
        norm_returns = (returns - returns.mean()) / (returns.std() + self.eps)

        """ CODE HERE:
                Now, we have the return for each time step and log probability for each time step.
                Then, we can compute the loss for each time step using the equation of updating \theta
                in the pseudocode. 
        """
        for log_prob, r in zip(self.saved_log_probs, norm_returns):
            # compute the loss for each time step

        # We sum all the policy loss
        policy_loss = torch.cat(policy_loss).sum()
        
        """ CODE HERE:
                Implement the one-step backpropagation
        """

        
        # after the backpropagation, clear the data to get ready to store the new episode data.
        del self.saved_log_probs[:]
        del self.saved_rewards[:]

        return returns[0].item(), policy_loss.item()

    def rollout(self):
        """ Function to collect one episode from the environment
        """
        
        """ CODE HERE:
                Implement the code to collect one episode. 
                
                Specifically, we only collect the rewards and corresponding log probability, which
                should be stored in "self.saved_rewards" and "self.saved_log_probs", respectively. 
                
                This is because we only need the return_t and log_probability to update the weights
                of the policy.
      
        """
        # reset the env

        # start rollout
            # render the action and log probability given a state (use the feature of the state)
            
            # save the log probability to "self.saved_log_probs"

            # render the next state, reward
            
            # save the reward to "self.saved_rewards"

            # check termination

    def run_train(self):
        # number of the training epsiode
        episode_num = self.params['episode_num']
        
        # list to store the returns and losses during the training
        train_returns = []
        train_losses = []
        
        # start the training
        ep_bar = tqdm.trange(episode_num)
        for ep in ep_bar:
            """CODE HERE
                    Implement the REINFORCE algorithm here.
            """
            # collect one episode

            # update the policy using the collected episode
            G, loss = None, None
            
            # save the returns and losses
            train_returns.append(G)
            train_losses.append(loss)
            
            # add description
            ep_bar.set_description(f"Episode={ep} | Discounted returns = {G} | loss = {loss:.2f}")
            
        # we have to smooth the returns for plotting
        smoothed_returns = rolling_average(np.array(train_returns), window_size=100)
        smoothed_losses = rolling_average(np.array(train_losses), window_size=100)
        
        return smoothed_returns, smoothed_losses

## Main function to run the training

In [ ]:
if __name__ == "__main__":
    my_env = FourRooms()

    train_params = {
        'episode_num': 2000,
        'learning_rate': 1e-3,
        'gamma': 0.99,
        'epsilon': 1e-6,

        'action_num': 4,
        'observation_dim': 3,

        'max_time_step_per_episode': 459
    }

    results_1_returns = []
    results_1_losses = []
    for _ in range(2):
        my_agent = REINFORCEAgent()

        my_trainer = REINFORCEAgentTrainer(my_agent, my_env, train_params)
        returns, losses = my_trainer.run_train()
        
        results_1_returns.append(returns)
        results_1_losses.append(losses)

# Plot the learning curves: discounted returns w.r.t epsiodes

In [ ]:
plot_curves([np.array(results_1_returns)],
            ['REINFORCE'],
            ['r'],
            'Discounted returns',
            "Discounted Returns Figure")

# Plot the loss curves: loss w.r.t epsiodes

In [ ]:
plot_curves([np.array(results_1_losses)], ['REINFORCE'], ['b'], 'Loss', "Loss Figure")

# Q3 : Implement the REINFORCE algorithm with baseline on page 330

Here, we will implement the REINFORCE algorithm with baseline on page 330 using the neural networks. 

Same as the ex8, we still use the Pytorch library to build the neural networks and train them.

Please read the pseudocode on page 330 carefully and complete the implementation below.

In [ ]:
class REINFORCEBaselineNet(nn.Module):
    def __init__(self):
        super(REINFORCEBaselineNet, self).__init__()

        """ CODE HERE:
                Implement the critic network here. The architecture should be as follows:
                
                Layer 1: Linear, input size 3, output size 128

                Activation 1: relu

                Layer 2: Linear, input size 128, output size 1

                Activation 2: identity
        """
        # critic network
        self.critic_network = None

        """ CODE HERE:
                Implement the critic network here. The architecture should be as follows:
                
                Layer 1: Linear, input size 3, output size 128

                Activation 1: relu

                Layer 2: Linear, input size 128, output size 4

                Activation 2: softmax
        """
        # actor network
        self.actor_network = None

    def forward(self, x):
        # compute the critic
        state_value = self.critic_network(x)

        # compute the actor
        action_probs = F.softmax(self.actor_network(x), dim=1)

        return state_value, action_probs

In [ ]:
class REINFORCEBaselineAgent(object):
    def __init__(self):
        # create the actor network and state value network
        self.policy_net = REINFORCEBaselineNet()

    def get_action(self, state):
        # derive an action, log probability, and the state value
        state_tensor = torch.tensor(state).float().view(1, -1)
        state_value, action_probs = self.policy_net(state_tensor)
        m = Categorical(action_probs)
        action = m.sample()
        log_prob = m.log_prob(action)
        return action.item(), log_prob, state_value

In [ ]:
class REINFORCEBaselineAgentTrainer(object):
    def __init__(self, agent, env, params):
        # Agent object
        self.agent = agent
        
        # Environment object
        self.env = env
        
        # Training parameters
        self.params = params

        # Lists to store the log probabilities, state values, and rewards for one episode
        self.saved_log_probs = []
        self.saved_state_values = []
        self.saved_rewards = []

        # Gamma
        self.gamma = params['gamma']

        # Small value for returns normalization
        self.eps = params['epsilon']

        # Create the optimizer
        """ CODE HERE:
                Implement the Adam optimizer with the learning rate in params
        """

    @staticmethod
    def compute_state_feature(state):
        return [state[0] / 10, state[1] / 10, 1]

    def update_agent_policy_network(self):
        # We define a list to store the policy loss for each time step
        policy_loss = []
        
        # We define a list to store the state value for each time step
        value_loss = []
        
        # We define a list to store the return for each time step
        returns = deque()

        """ CODE HERE:
                Compute the return for each time step
                
                Hint: We usually compute the return from the end. Remember to append it 
                      correctly. You can use "returns.appendleft(G)"
        """
        # compute returns for every time step

        # normalize the returns
        returns = torch.tensor(returns)
        norm_returns = (returns - returns.mean()) / (returns.std() + self.eps)

        """ CODE HERE:
                Now, we have the return, state value, and log probability for each time step.
                Then, we can compute the loss for each time step using the equation of updating \theta
                in the pseudocode. 
        """
        for log_prob, val, r in zip(self.saved_log_probs, self.saved_state_values, norm_returns):
            # compute the policy loss for each time step

        # compute the total loss
        total_loss = torch.stack(policy_loss).sum() + torch.stack(value_loss).sum()

        # optimization
        self.optim.zero_grad()
        total_loss.backward()
        self.optim.step()

        del self.saved_log_probs[:]
        del self.saved_state_values[:]
        del self.saved_rewards[:]

        return returns[0].item(), total_loss.item()

    def rollout(self):
        """ CODE HERE:
                Implement the code to collect one episode. 
                
                Specifically, we only collect the rewards and corresponding log probability, which
                should be stored in "self.saved_rewards" and "self.saved_log_probs", respectively. 
                
                This is because we only need the return_t and log_probability to update the weights
                of the policy.
      
        """
        # reset the env

        # start rollout
            # render the action and log probability given a state (use the feature of the state)
            
            # save the log probability to "self.saved_log_probs"
            
            # save the state value to "self.saved_state_values"

            # render the next state, reward
            
            # save the reward to "self.saved_rewards"

            # check termination

    def run_train(self):
        # number of the training epsiode
        episode_num = self.params['episode_num']
        
        # list to store the returns and losses during the training
        train_returns = []
        train_losses = []
        
        ep_bar = tqdm.trange(episode_num)
        for ep in ep_bar:  
            """CODE HERE
                    Implement the REINFORCE algorithm here.
            """
            # collect one episode

            # update the policy using the collected episode

            # update the policy using the collected episode
            
            # save the returns and losses
            train_returns.append(G)
            train_losses.append(loss)

            ep_bar.set_description(f"Episode={ep} | Discounted returns = {G} | loss = {loss:.2f}")
            
        # we have to smooth the returns for plotting
        smoothed_returns = rolling_average(np.array(train_returns), window_size=100)
        smoothed_losses = rolling_average(np.array(train_losses), window_size=100)
        
        return smoothed_returns, smoothed_losses

In [ ]:
if __name__ == "__main__":
    my_env = FourRooms()

    train_params = {
        'episode_num': 2000,
        'learning_rate': 1e-3,
        'gamma': 0.99,
        'epsilon': 1e-6,

        'action_num': 2,
        'observation_dim': 4,

        'max_time_step_per_episode': 459,

        'input_dim': 3,
        'hidden_dim': 128,
        'hidden_num': 1,
        'output_dim': 4
    }

    results_2_returns = []
    results_2_losses = []
    for _ in range(2):
        my_agent = REINFORCEBaselineAgent()
        my_trainer = REINFORCEBaselineAgentTrainer(my_agent, my_env, train_params)
        returns, losses = my_trainer.run_train()
        
        results_2_returns.append(returns)
        results_2_losses.append(losses)

# Plot the learning curves: discounted returns w.r.t epsiodes

In [ ]:
plot_curves([np.array(results_2_returns)], ['REINFORCE-baseline'], ['r'], 'Discounted returns', "Discounted Returns Figure")

# Plot the loss curves: loss w.r.t epsiodes

In [ ]:
plot_curves([np.array(results_2_losses)], ['REINFORCE-baseline'], ['b'], 'Loss', "Loss Figure")

# Q3 - Compare vanilla REINFORCE algorithm v.s. REINFORCE with baseline.

# Compare learning curves : discounted returns w.r.t epsiodes

In [ ]:
plot_curves([np.array(results_1_returns), np.array(results_2_returns)], 
            ['REINFORCE', 'REINFORCE-baseline'], 
            ['r', 'b'], 'Discounted returns', "Discounted Returns Figure")

# Compare loss curves : loss w.r.t epsiodes

In [ ]:
plot_curves([np.array(results_1_losses), np.array(results_2_losses)], 
            ['REINFORCE', 'REINFORCE-baseline'], 
            ['r', 'b'], 'Loss', "Loss Figure")